This ipynb shows an example of using `get_1site` API to solve the "1-site-N-wells" problem, or the "site-level" layout optimization.  
"1-site-N-wells" problem means to find the best drill site location with optimized wellbore trajectories fulfilling various constraints,  
so that the total cost(length) of the N wellbores is minimum.  

In this example, there are 4 wells to be drilled from 1 site.  
The cost of a wellbore is simplified as its length.  
The only constraint is the dogleg severity.  

API url:  
url = "https://home-test.make234.com/api/v1/get_1site"

# 0. Initiate environment

In [1]:
import requests
import json
import numpy as np
from plotly import graph_objects as go
import time

In [2]:
import sys
import os
rootpath=os.getcwd().split("WelLayout_API") [0] + "\WelLayout_API"
print(rootpath)
sys.path.append(rootpath)

from tools.input2json import input2json
from tools.PlotSurvey_plotly import PlotSurvey_plotly as PlotSurvey
from tools.PlotContour_plotly import PlotContour_plotly as PlotContour

d:\OneDrive\OneDrive - LHG\Projects\\WelLayout_API


# 1. Prepare Data

## 1.1 Target information
The completion intervals (targets) are normally decided by the reservoir engineers based on the reservoir layer's trend to maximize the total production of the wells.  
Here we need the location of the first point of the completion interval($PT$) and its desired entry direction($VT$) for each well.

Following are the data for 4 targets. Multiple wells' information is gathered in a Matrix.

In [3]:
# 4 wells to be designed
n=4

# location of the first point of the completion interval (PT), [X, Y, Z]
PTM= np.array ([[517167.9, 6782513.9, -1550.0], 
                [518255.3, 6784864.0, -1559.0],
                [516179.0, 6782799.0, -1557.0],
                [515726.0, 6784079.0, -1577.0]])

# drilling direction at the first point of the completion interval (VT),
VTM= np.array ([[968.2, 636.1, -14.0],
                [-851.30, 692.0, -1.0],
                [258.0, 1055.0, -23.0],
                [541.0, 1051.0, -4.0]])


## 1.2 KOP information
The KOP depth is normally determined by the drilling engineer based on the rock mechanical property of the formation to ensure the wellbore stabilty.  
Normally, the direction at KOP is vertically downwards.

In [4]:
# location of the KOP (PK), [X, Y, Z]
# set X, Y as np.nan, so that the API can find the best site location
# KOP is beneath site
PKM= np.array ([[np.nan, np.nan, -400.0],
                [np.nan, np.nan, -400.0],
                [np.nan, np.nan, -400.0],
                [np.nan, np.nan, -400.0]])

# drilling direction at the KOP (VK), [X, Y, Z]
VKM= np.array ([[0.0, 0.0, -1.0],
                [0.0, 0.0, -1.0],
                [0.0, 0.0, -1.0],
                [0.0, 0.0, -1.0]])

## 1.3 DLS constraint
Dogleg severity (DLS) is constrained by the drilling equipment.  
Drilling engineers need to set a maximum allowable DLS to ensure the feasibility for drilling and the wellbore stability.  
DLS unit is given in `°/30m`

In the site-level case, we adopt the simplest wellbore trajectory where there are at most 2 curved sections, one at the buildup section after the KOP and the other at the landing section before the Target.

In [5]:
# DLS at the buildup section after the KOP 
dls_KOP=2.0

# DLS at the landing section before the Target
dls_Target=3.5

# format DLS for API
DLSM= np.array ([[dls_KOP, dls_Target],
                [dls_KOP, dls_Target],
                [dls_KOP, dls_Target],
                [dls_KOP, dls_Target]])

# optional: get the corresponding curvature radius of the DLS
rM= 30*180/DLSM/np.pi
print(rM)

[[859.4366927  491.10668154]
 [859.4366927  491.10668154]
 [859.4366927  491.10668154]
 [859.4366927  491.10668154]]


## 1.4 [Optional] Other constraints
A constraint function is a function involving the parameters:  
`PK`, KOP location, 3D.  
`PT`, target location, 3D.  
`angDK`, turning angle(°) of the buildup section after the KOP, scalar.  
`angDT`, turning angle(°) of the landing section before the target, scalar.  
`P_L`, point on the trajectory at the given depth, 3D.  
`inclD_L`, inclination(°) of the trajectory at the given depth, scalar, [0, 180).  
`aziD_L`, azimuth(°) of the trajectory at the given depth, scalar, [0,360).  
...  
(**contact the author for more parameters related to the trajectory structure to build more complex engineering constraints**)  

Constraints for each well are gathered in a list.
All constraint functions are gathered in `neconM`.

$neconM \ge 0$

In this example, we do not have any other constraint.

In [6]:
neconM=None

### 1.4.1 constraints for site location
The site location constraint is a function of [X, Y] of the site 2D coordinate.  
We simplify all wells' KOP points are the same, and exactly beneath the site location, hence the site location can be presented by $PK$.  
X: `PK[0]`  
Y: `PK[1]`  

As for the 1-site-N-wells problem, all wells are drilled from the same site, hence the site location constraints for one well are also active for the others.
This means you just need to write the constraint functions for one well only.

In [7]:
# # uncomment the cell for site location constraints
# # The following constraints mean that the site location [X, Y] must fulfill X<516000, Y<6782000
# neconM=[
#         ["-PK[0]+516000", "-PK[1]+6782000"], # constraints for the 1st wellbore
#         None, # constraints for the 2nd wellbore
#         None, # constraints for the 3rd wellbore
#         None  # constraints for the 4th wellbore
#         ] 



In [8]:
# # same effect as above
# neconM=[
#         ["-PK[0]+516000", "-PK[1]+6782000"], # constraints for the 1st wellbore
#         ["-PK[0]+516000", "-PK[1]+6782000"], # constraints for the 2nd wellbore
#         None, # constraints for the 3rd wellbore
#         None  # constraints for the 4th wellbore
#         ]

### 1.4.2 constraints for trajectory tortuosity

The trajectory tortuosity constraint is a function of `angDK` and `angDT`.  
Limiting the turning angles in the trajectory is critical for controlling the torque & drag.

In [ ]:
# # uncomment the cell for tortuosity constraints
# # The following constraints mean that for the 2nd wellbore, the turning angle of the landing section must be smaller than 90°

# neconM=[
#         None, # constraints for the 1st wellbore
#         ["-angDT+90"], # constraints for the 2nd wellbore
#         None, # constraints for the 3rd wellbore
#         None # constraints for the 4th wellbore
#         ]

### 1.4.3 constraints at certain depths (layers)

The layer constraint is a function of `P_L`, `inclD_L` and `aziD_L`.  
For each layer, the contraints are collected in a dictionary of two keys: `layer` and `con`.  
`layer` is a scalar depth value， or scattered 3D points (not recommended, as it's time consuming to deal discretized data in optimization).  
`con` is a list of constraint functions.

In [10]:
lay_conM=None

In [11]:
# # uncomment the cell for layer constraints
# The following constraint means that for the 2nd wellbore, the inclination angle at the depth of -1300m must be smaller than 81°

# lay_conM=[
#         None, # constraints for the 1st wellbore
#         [{'layer': -1300, 'con': ["-inclD_L+81"]}], # constraints for the 2nd wellbore
#         None, # constraints for the 3rd wellbore
#         None # constraints for the 4th wellbore
#         ]

# 1.5 [Optional] Objective(cost) function
User-defined cost function for the trajectory, default is the length of the trajectory.


In [12]:
ObjM=None
# 
# ObjM=["L", "L", "L", "L"] # same a default


You can also define any cost function for the trajectory.  
The function is related with `Ls`, `Lc` and `incl`.  
`Ls` is the length of the straight section,   
`Lc` is the length of the curved section,  
`incl` is the inclination angle (rad) of the straight section between the two curved section, rad.

In [13]:
# # uncomment the following lines to see different results
# ObjM=["2*Lc+Ls*(1+np.sin(incl))", # Lc: length of the curved section.
#       "2*Lc+Ls*(1+np.sin(incl))", # Ls: length of the straight section.
#       "2*Lc+Ls*(1+np.sin(incl))", 
#       "2*Lc+Ls*(1+np.sin(incl))"] 

In [14]:
# # uncomment the following lines to see different results
# ObjM=["2*Lc+Ls", # Lc: length of the curved section.
#       "2*Lc+Ls", # Ls: length of the straight section.
#       "2*Lc+Ls", 
#       "2*Lc+Ls"] 

# 1.6 [Optional] Computational parameters 
Normally, you do not need to bother yourself about these paramters. API will automatically apply default values.

In [ ]:
# discretized measured depth for trajectory data
MD_intervalM= None, # shape (n, ), default: 30m

In [ ]:
# plot canvas
XRange= None, # shape (2, ), default: [min(X) of given points(PKM, PTM) - max(rM),  max(X) of given points(PKM, PTM) + max(rM)]
YRange= None, # shape (2, )

In [17]:
# grid resolution for cost contour
resolution= None, # scalar, default is 50

In [18]:
# radius of the cost contour for each well
cst_radiusM= None, # shape (n, ), default is 3000

# 2. Use the API to do the computation

## 2.1 save the input data to JSON

In [19]:
# save to current path, file name "input.json"
filepath="input.json"

input_data=input2json(n,                
        PTM,
        VTM,
        PKM,
        VKM,
        DLSM,
        ObjM=ObjM,
        neconM=neconM,
        lay_conM=lay_conM,

        MD_intervalM=MD_intervalM,
        XRange=XRange,
        YRange=YRange,
        resolution=resolution,
        cst_radiusM=cst_radiusM,
        
        filepath=filepath)

=====file input.json written successfully=====


## 2.2 prepare the parameters for calling API

In [20]:
# load input data (json) for computation
with open(filepath) as json_file:
    input_data = json.load(json_file)

# Set request headers
headers = {
    'Content-Type': 'application/json'
}

# API url
url ="https://home-test.make234.com/api/v1/get_1site"

## 2.3 fetch the response from the API

This case take about 15 seconds to get the result.  
The result includes the 4 trajectories and 1 cost contour.

In [ ]:
tic=time.time()
try:
    response = requests.post(url, json=input_data, headers=headers, timeout=300)
    
    # Check response status
    response.raise_for_status()
    
    # Print response results
    print("Status code:", response.status_code)
    print("Response content:")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))

    # Save response content to JSON file
    with open('output.json', 'w', encoding='utf-8') as f:
        json.dump(response.json(), f, indent=2, ensure_ascii=False)
    print("Response content has been saved to output.json")
    print(url)
    
except requests.exceptions.Timeout:
    print("Request timeout (exceeded 5 minutes)")
except requests.exceptions.RequestException as e:
    print("Request error:", e)
except json.JSONDecodeError:
    print("Response is not valid JSON format")
    print("Raw response:", response.text)
toc=time.time()
print(f"elapsed time: {toc-tic} s")

Status code: 200
Response content:
{
  "status": "success",
  "data": {
    "Curves": [
      {
        "MD": [
          0.0,
          30.0,
          60.0,
          90.0,
          120.0,
          150.0,
          180.0,
          210.0,
          240.0,
          270.0,
          300.0,
          330.0,
          360.0,
          390.0,
          400.0,
          420.0,
          450.0,
          480.0,
          510.0,
          540.0,
          570.0,
          600.0,
          630.0,
          660.0,
          690.0,
          720.0,
          750.0,
          770.718471317432,
          780.0,
          810.0,
          840.0,
          870.0,
          900.0,
          930.0,
          960.0,
          990.0,
          1020.0,
          1050.0,
          1080.0,
          1110.0,
          1140.0,
          1170.0,
          1200.0,
          1230.0,
          1260.0,
          1272.5827085522908,
          1290.0,
          1320.0,
          1350.0,
          1380.0,
      

# 3. Visualize the results

## 3.1 load the output JSON file

In [22]:
# Load the output JSON file
with open("output.json", "r") as file:
    data = json.load(file)

# Extract the trajectory data 
wellbores = data["data"]["Curves"]

# Cost contour data
contour_1site=data['data']['CostASite']
for key in contour_1site.keys():
    contour_1site[key]=np.array(contour_1site[key], dtype=float)

## 3.1 check the data

In [23]:
# Loop through each wellbore
for wellbore in wellbores:
    # check the target location in the ouput
    print([wellbore['X'][-1], wellbore['Y'][-1], wellbore['Z'][-1]])

# check the target location in the input
input_data['FIELDOPT INPUT BLOCK']['PTM']

[517167.9, 6782513.9, -1550.0]
[518255.3, 6784864.0, -1559.0]
[516179.0, 6782799.0, -1557.0]
[515726.0, 6784079.0, -1577.0]


{'DESCRIPTION': 'target location, i.e., the 1st point of completion interval. 3D, [EAST,NORTH,Depth]',
 'UNIT': 'm',
 'VALUE': [[517167.9, 6782513.9, -1550.0],
  [518255.3, 6784864.0, -1559.0],
  [516179.0, 6782799.0, -1557.0],
  [515726.0, 6784079.0, -1577.0]]}

## 3.2 3D Visualization
Plot the wellbore trajectories and the cost contour for the site.  
The cost contour indicates the total cost to reach all the targets while placing the drill site at the given [X, Y] location.  
If the cost contour at a given point is empty, then it means the site located at this point cannot reach at least one of the targets under the given constraints.

In [24]:
# Plot wellbore trajectory
fig1=None #initiate a fig
styles=['k-', 'r-', 'g-', 'b-'] # line style for each trajectory
for i, wellbore in enumerate(wellbores):
    fig1=PlotSurvey(wellbore, # trajectory data
                    fig=fig1, # figure handle
                    style=styles[i], # line style for the trajectory
                    name=f'Well #{i+1}', # name of the trajectory in the plot
                    show=0) # wait for plottiing all trajectories

# plot the cost contour for the site
PlotContour(X=contour_1site['X'],
            Y=contour_1site['Y'],
            Contour_Val=contour_1site['cost'],
            fig=fig1,
            name='cost contour',
            azim=-65, # view angle azimuth
            elev=40, # view angle elevation
            show=0)
mincost=np.nanmin(contour_1site['cost'])
try:
    minidx=np.nanargmin(contour_1site['cost'])
except:
    minidx=None
print (f"min cost (node value): {mincost}")
print (f"min cost (node location): {[contour_1site['X'][minidx], contour_1site['Y'][minidx]]}")

# show the figure
fig1.show()

min cost (node value): 9297.797612530981
min cost (node location): [516500.0, 6782400.0]


In [25]:
figpath="figure.html"
fig1.update_layout(
            autosize=True,  # auto resize the fig1 in webbrowers
            width=None,
            height=None,
            margin=dict(l=0, r=0, t=0, b=0),)
fig1.write_html(figpath, full_html=True, div_id="plotly-div", config={"responsive": True})
print(f"figure saved to file: {figpath}")

figure saved to file: figure.html
